In [1]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy

In [2]:
data = pd.read_csv("..//..//dataset//oakland-crime-statistics-2011-to-2016//records-for-2014.csv")
print(data)

       Agency              Create Time  Area Id Beat  Priority  \
0          OP  2014-01-01T00:00:00.000      1.0  02X         2   
1          OP  2014-01-01T00:00:00.000      2.0  26Y         2   
2          OP  2014-01-01T00:00:00.000      2.0  30Y         2   
3          OP  2014-01-01T00:00:00.000      2.0  30Y         2   
4          OP  2014-01-01T00:01:04.000      2.0  35X         2   
5          OP  2014-01-01T00:01:08.000      2.0  32X         2   
6          OP  2014-01-01T00:01:23.000      2.0  31Z         2   
7          OP  2014-01-01T00:01:31.000      2.0  23X         2   
8          OP  2014-01-01T00:01:40.000      1.0  02Y         2   
9          OP  2014-01-01T00:02:10.000      2.0  35X         2   
10         OP  2014-01-01T00:03:10.000      2.0  29X         2   
11         OP  2014-01-01T00:04:13.000      1.0  06X         2   
12         OP  2014-01-01T00:04:34.000      2.0  23X         2   
13         OP  2014-01-01T00:04:48.000      2.0  32Y         2   
14        

#### 1. 删除空值处理

In [3]:
#删除所有属性中有空值的行
data1 = data.dropna()
print(data1)

Empty DataFrame
Columns: [Agency, Create Time, Area Id, Beat, Priority, Incident Type Id, Incident Type Description, Event Number, Closed Time, Location 1, Zip Codes]
Index: []


#### 2. 用最高频率值进行填充

In [4]:
data2 = copy.deepcopy(data)#拷贝原数据集用于填充
for i in range(10):
    count = data2.iloc[:,i].value_counts()  #依次获取每一列的series
    name = pd.DataFrame(data2.iloc[:,i]).columns.values.tolist()
    sort_ans = count.sort_values(ascending = False) #对各类别的频数值进行排序 并选择最高频率的索引
    fre = sort_ans.index[0]
    fre = str(fre)
    print(name[0]+"的最高频率值为"+fre+'\n')
    
    data2[name[0]] = data2[name[0]].fillna(fre) #进行缺失值的填充
print("------------------------------------------"+'\n'+"填充完毕")

Agency的最高频率值为OP

Create Time的最高频率值为2014-10-14T02:45:12.000

Area Id的最高频率值为1.0

Beat的最高频率值为04X

Priority的最高频率值为2

Incident Type Id的最高频率值为933R

Incident Type Description的最高频率值为ALARM-RINGER

Event Number的最高频率值为LOP140627000103

Closed Time的最高频率值为2014-06-04T16:31:09.000

Location 1的最高频率值为{'human_address': '{"address": "INTERNATIONAL BLVD", "city": "", "state": "", "zip": ""}'}

------------------------------------------
填充完毕


In [5]:
print(data2)  #填充结果

       Agency              Create Time Area Id Beat  Priority  \
0          OP  2014-01-01T00:00:00.000       1  02X         2   
1          OP  2014-01-01T00:00:00.000       2  26Y         2   
2          OP  2014-01-01T00:00:00.000       2  30Y         2   
3          OP  2014-01-01T00:00:00.000       2  30Y         2   
4          OP  2014-01-01T00:01:04.000       2  35X         2   
5          OP  2014-01-01T00:01:08.000       2  32X         2   
6          OP  2014-01-01T00:01:23.000       2  31Z         2   
7          OP  2014-01-01T00:01:31.000       2  23X         2   
8          OP  2014-01-01T00:01:40.000       1  02Y         2   
9          OP  2014-01-01T00:02:10.000       2  35X         2   
10         OP  2014-01-01T00:03:10.000       2  29X         2   
11         OP  2014-01-01T00:04:13.000       1  06X         2   
12         OP  2014-01-01T00:04:34.000       2  23X         2   
13         OP  2014-01-01T00:04:48.000       2  32Y         2   
14         OP  2014-01-01

#### 3. 根据数据对象的相似性来填充数据值

   这里我们使用的是KNN算法，实体之间的距离计算使用曼哈顿距离。K的值设置为6。我们选取与存在空缺值的向量距离最近的6个向量在对应属性上的平均值来对空缺值进行填充。